In [11]:
import pickle as pickle
import os
import pandas as pd
import torch
from sklearn.metrics import accuracy_score
from transformers import AutoTokenizer, BertForSequenceClassification, Trainer, TrainingArguments, BertConfig
from load_data import *



data = load_data("/opt/ml/input/data/train/train.tsv")
data.head()

,sentence,entity_01,entity_02,label
0,영국에서 사용되는 스포츠 유틸리티 자동차의 브랜드로는 랜드로버(Land Rover)...,랜드로버,자동차,17
1,"선거에서 민주당은 해산 전 의석인 230석에 한참 못 미치는 57석(지역구 27석,...",민주당,27석,0
2,유럽 축구 연맹(UEFA) 집행위원회는 2014년 1월 24일에 열린 회의를 통해 ...,유럽 축구 연맹,UEFA,6
3,"용병 공격수 챠디의 부진과 시즌 초 활약한 강수일의 침체, 시즌 중반에 영입한 세르...",강수일,공격수,2
4,람캄행 왕은 1237년에서 1247년 사이 수코타이의 왕 퍼쿤 씨 인트라팃과 쓰엉 ...,람캄행,퍼쿤 씨 인트라팃,8


In [3]:
data.groupby(['label']).count()

,sentence,entity_01,entity_02
label,,,
0,4440,4440,4440
1,92,92,92
2,553,553,553
3,44,44,44
4,679,679,679
5,186,186,186
6,231,231,231
7,209,209,209
8,164,164,164


In [4]:
import pickle

with open('/opt/ml/input/data/label_type.pkl', 'rb') as f:
    label_type = pickle.load(f)
label_type

{'관계_없음': 0,
 '인물:배우자': 1,
 '인물:직업/직함': 2,
 '단체:모회사': 3,
 '인물:소속단체': 4,
 '인물:동료': 5,
 '단체:별칭': 6,
 '인물:출신성분/국적': 7,
 '인물:부모님': 8,
 '단체:본사_국가': 9,
 '단체:구성원': 10,
 '인물:기타_친족': 11,
 '단체:창립자': 12,
 '단체:주주': 13,
 '인물:사망_일시': 14,
 '단체:상위_단체': 15,
 '단체:본사_주(도)': 16,
 '단체:제작': 17,
 '인물:사망_원인': 18,
 '인물:출생_도시': 19,
 '단체:본사_도시': 20,
 '인물:자녀': 21,
 '인물:제작': 22,
 '단체:하위_단체': 23,
 '인물:별칭': 24,
 '인물:형제/자매/남매': 25,
 '인물:출생_국가': 26,
 '인물:출생_일시': 27,
 '단체:구성원_수': 28,
 '단체:자회사': 29,
 '인물:거주_주(도)': 30,
 '단체:해산일': 31,
 '인물:거주_도시': 32,
 '단체:창립일': 33,
 '인물:종교': 34,
 '인물:거주_국가': 35,
 '인물:용의자': 36,
 '인물:사망_도시': 37,
 '단체:정치/종교성향': 38,
 '인물:학교': 39,
 '인물:사망_국가': 40,
 '인물:나이': 41}

In [5]:
# load model and tokenizer
MODEL_NAME = "bert-base-multilingual-cased"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

# load dataset
train_dataset = load_data("/opt/ml/input/data/train/train.tsv")
#dev_dataset = load_data("./dataset/train/dev.tsv")
train_label = train_dataset['label'].values
#dev_label = dev_dataset['label'].values

# tokenizing dataset
tokenized_train = tokenized_dataset(train_dataset, tokenizer)
#tokenized_dev = tokenized_dataset(dev_dataset, tokenizer)

랜드로버[SEP]자동차
민주당[SEP]27석
유럽 축구 연맹[SEP]UEFA


In [6]:
tokenized_train

{'input_ids': tensor([[   101,   9167,  15001,  ...,      0,      0,      0],
        [   101,   9311,  16323,  ...,      0,      0,      0],
        [   101,  68495,  37905,  ...,      0,      0,      0],
        ...,
        [   101,   9328, 118782,  ...,    123, 101322,    102],
        [   101,  49780,  16617,  ...,      0,      0,      0],
        [   101,   9730,  30858,  ...,      0,      0,      0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 1, 1, 1],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}

In [8]:
train_label

array([17,  0,  6, ...,  2,  0,  2])

In [9]:
train_dataset

,sentence,entity_01,entity_02,label
0,영국에서 사용되는 스포츠 유틸리티 자동차의 브랜드로는 랜드로버(Land Rover)...,랜드로버,자동차,17
1,"선거에서 민주당은 해산 전 의석인 230석에 한참 못 미치는 57석(지역구 27석,...",민주당,27석,0
2,유럽 축구 연맹(UEFA) 집행위원회는 2014년 1월 24일에 열린 회의를 통해 ...,유럽 축구 연맹,UEFA,6
3,"용병 공격수 챠디의 부진과 시즌 초 활약한 강수일의 침체, 시즌 중반에 영입한 세르...",강수일,공격수,2
4,람캄행 왕은 1237년에서 1247년 사이 수코타이의 왕 퍼쿤 씨 인트라팃과 쓰엉 ...,람캄행,퍼쿤 씨 인트라팃,8
...,...,...,...,...
8995,2002년 FIFA 월드컵 사우디아라비아와의 1차전에서 독일은 8-0으로 승리하였는...,사우디아라비아,2002년,0
8996,일본의 2대 메이커인 토요타와 닛산은 시장 점유율을 높이기 위한 신차 개발을 계속하...,토요타,일본,9
8997,방호의의 손자 방덕룡(方德龍)은 1588년(선조 21년) 무과에 급제하고 낙안군수로...,방덕룡,선무원종공신(宣武原從功臣),2
8998,LG전자는 올해 초 국내시장에 출시한 2020년형 ‘LG 그램’ 시리즈를 이달부터 ...,LG전자,북미,0


In [10]:
print('Tokenized: ', tokenizer.tokenize(train_dataset['sentence'][0]))

Tokenized:  ['영국', '##에서', '사', '##용', '##되는', '스', '##포츠', '유', '##틸', '##리', '##티', '자', '##동차', '##의', '브', '##랜드', '##로는', '랜', '##드', '##로', '##버', '(', 'Land', 'Rover', ')', '와', '지', '##프', '(', 'Je', '##ep', ')', '가', '있으며', ',', '이', '브', '##랜드', '##들은', '자', '##동차', '##의', '종', '##류', '##를', '일', '##컫', '##는', '말', '##로', '사', '##용', '##되', '##기도', '한다', '.']
